# **How to build RAG application - PDF method**
---
Demo how to create RAG from PDF file 

## **Library Installation**
Install those required libary

In [1]:
%pip install --quiet -U langchain ## LLM libary
%pip install --quiet -U chromadb ## Vector Storage
# %pip install --quiet -U langchain-chroma ## LLM Vector Storage
%pip install --quiet -U pypdf ## Loading PDFs
%pip install --quiet -U pytest ## Unit testing
%pip install --quiet -U langchain-community ## LLM Community Library
%pip install --quiet -U langchain-ollama ## LLM Ollama Library

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Import some libraries

In [2]:
import argparse
import os
import shutil
from IPython.display import display, Markdown
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema.document import Document
from langchain_ollama import OllamaEmbeddings
from langchain.vectorstores.chroma import Chroma
# from langchain_chroma import Chroma
from langchain.prompts import ChatPromptTemplate
# from langchain_community.llms.ollama import Ollama
from langchain_ollama import OllamaLLM

Define variables

In [3]:
# PDF_PATH = "./data/pdf/Maestro_Policy_Engine_25.40.00_Configuration_Guide.pdf"
# CHROMA_PATH = "./chroma-database/mpe-db"
# PDF_PATH = "./data/pdf/Monopoly Manual 2007.pdf"
# CHROMA_PATH = "./chroma-database/monopoly-db"
PDF_PATH = "./data/pdf/mysql-tutorial-excerpt-8.0-en.a4.pdf"
CHROMA_PATH = "./chroma-database/mysql-db"

## **Loading PDF Data**
Try to load PDF data

In [4]:

loader = PyPDFLoader(PDF_PATH)
pages = loader.load()
# print(pages)

Split the documents

In [5]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=80,
    length_function=len,
    is_separator_regex=False
)
chunks = text_splitter.split_documents(pages)

Preparing embedding

In [6]:
embeddings = OllamaEmbeddings(model="nomic-embed-text")

**(OPTIONAL) Clear Databse**
Clear database if required

In [7]:
IS_DB_CLEARED = False
if IS_DB_CLEARED:
     if os.path.exists(CHROMA_PATH):
        shutil.rmtree(CHROMA_PATH)

## **Preparing Chroma**

In [8]:
db = Chroma(
    persist_directory=CHROMA_PATH,
    embedding_function=embeddings
)

/tmp/ipykernel_14307/2609319021.py:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(


Calculate Page IDs

In [9]:
last_page_id = None
current_chunck_index = 0

# Calculate the page ID
print(f"Processing {len(chunks)} chunks")
for chunk in chunks:
    source = chunk.metadata.get("source")
    page = chunk.metadata.get("page")
    current_page_id = f"{source}:{page}"
    # print(f"=== Processing {current_page_id} ===")

    # if the page ID is the same as the last one, increment the index
    # print(f"Last page ID: {last_page_id} | Current page ID: {current_page_id}")

    if current_page_id == last_page_id:
        current_chunck_index += 1
    else:
        current_chunck_index = 0
    # print(f"Chunk index: {current_chunck_index}")

    # Calculate the chunk ID
    chunk_id = f"{current_page_id}:{current_chunck_index}"
    last_page_id = current_page_id
    # print(f"Chunk ID: {chunk_id}")

    # Add it to the page meta-data
    chunk.metadata["id"] = chunk_id

Processing 158 chunks


Add or update the documents

In [10]:
existing_items = db.get(include=[]) # IDs are always included by default
existing_ids = set(existing_items["ids"])
print(f"Number of existing documents in DB: {len(existing_ids)}")

Number of existing documents in DB: 158


Only add document that don't exist in the DB.

In [11]:
new_chunks = []
for chunk in chunks:
    if chunk.metadata["id"] not in existing_ids:
        new_chunks.append(chunk)

if len(new_chunks) > 0:
    print(f"Adding new documents to DB: {len(new_chunks)}")
    chunks = [chunk.metadata["id"] for chunk in new_chunks]
    db.add_documents(new_chunks, ids=chunks)
    db.persist()
else:
    print("No new documents to add to DB")

No new documents to add to DB


## **Query Data**
Preparing to search from DB

In [25]:
# reuse existing db
query_text = """
Imagine you are Database Engineer who has been given several tasks as below:
- Create a new table to store information of the company staffs in MySQL database server. It will contain the "First Name", "Last Name", "Age", "Contact Number", and "Email".
- Insert a few records in table that just created.
- Select all record in table that just inserted.
You are requested to create a document to share with your team member.
Please provide some examples of the SQL query.\n
Could you please provide the steps in bullet points.\n
Sample as below:\n
Step 1: ...\n
Step 2: ...\n
Step 3: ...\n
"""
PROMPT_TEMPLATE = """
1. If not sure, say "I don't know".
2. Answer the question based only on the following context:

Context: {context}

---

Answer the question based on the above context: {question}
"""
results = db.similarity_search_with_score(query_text, k=5)
context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
print(f"Context: \n{context_text}")
print(f"Question: \n{query_text}")
prompt = prompt_template.format(context=context_text, question=query_text)

Context: 
Creating a Table
Note
You can see at any time which database is currently selected using SELECT
DATABASE().
4.2 Creating a Table
Creating the database is the easy part, but at this point it is empty, as SHOW TABLES tells you:
mysql> SHOW TABLES;
Empty set (0.00 sec)
The harder part is deciding what the structure of your database should be: what tables you need and
what columns should be in each of them.
You want a table that contains a record for each of your pets. This can be called the pet table, and
it should contain, as a bare minimum, each animal's name. Because the name by itself is not very
interesting, the table should contain other information. For example, if more than one person in your

---

operations:
• Create a database
• Create a table
• Load data into the table
• Retrieve data from the table in various ways
• Use multiple tables
The menagerie database is simple (deliberately), but it is not difficult to think of real-world situations
in which a similar type o

Split the reasoning if deepseek LLM called

In [26]:
def extract_think_content(response_text):
    """
    Parses and separates content wrapped in XML-style "think" tags
    from the final response.
    """
    start_tag = "<think>"
    end_tag = "</think>"

    start_index = response_text.find(start_tag) + len(start_tag)
    end_index = response_text.find(end_tag)

    if start_index != -1 and end_index != -1:
        reasoning_content = response_text[start_index:end_index].strip()
        final_response = response_text[end_index + len(end_tag):].strip()
        return reasoning_content, final_response
    else:
        return None, response_text

Preparing Model for prompt

In [28]:
# MODEL_NAME = "deepseek-r1:1.5b"
# MODEL_NAME = "deepseek-r1:8b"
MODEL_NAME = "llama3.2:3b"
# MODEL_NAME = "tinyllama"
# MODEL_NAME = "tinydolphin"
# MODEL_NAME = "phi3"

model = OllamaLLM(model=MODEL_NAME)
response_text = model.invoke(prompt)

sources = [doc.metadata.get("id", None) for doc, _score in results]
if MODEL_NAME == "deepseek-r1:8b" or MODEL_NAME == "deepseek-r1:1.5b":
    reasoning_content, final_response = extract_think_content(response_text)
    formatted_response = f"Query:\n\n{query_text}\n\nResponse:\n\n{final_response}\n\nReasoning:\n\n{reasoning_content}\n\nSources: {sources}"
else:
    formatted_response = f"Query:\n\n{query_text}\n\nResponse:\n\n{response_text}\n\nSources: {sources}"
# print(formatted_response)
display(Markdown(formatted_response))

Query:


Imagine you are Database Engineer who has been given several tasks as below:
- Create a new table to store information of the company staffs in MySQL database server. It will contain the "First Name", "Last Name", "Age", "Contact Number", and "Email".
- Insert a few records in table that just created.
- Select all record in table that just inserted.
You are requested to create a document to share with your team member.
Please provide some examples of the SQL query.

Could you please provide the steps in bullet points.

Sample as below:

Step 1: ...

Step 2: ...

Step 3: ...



Response:

I don't know. 

However, based on the context provided, here is a possible solution for the given tasks:

**Task 1: Create a new table to store information of the company staffs**

* Create a database called "staff" (as per the context, creating a database is the easy part)
* Create a table called "staff_info" with the following columns: First Name, Last Name, Age, Contact Number, and Email
* Use the following SQL query to create the table:
```sql
CREATE TABLE staff_info (
  first_name VARCHAR(255),
  last_name VARCHAR(255),
  age INT,
  contact_number VARCHAR(20),
  email VARCHAR(255)
);
```

**Task 2: Insert a few records in the table**

* Use the following SQL query to insert a record into the "staff_info" table:
```sql
INSERT INTO staff_info (first_name, last_name, age, contact_number, email)
VALUES ('John', 'Doe', 30, '1234567890', 'john.doe@example.com');
```
You can repeat this query multiple times to insert more records.

**Task 3: Select all records in the table**

* Use the following SQL query to select all records from the "staff_info" table:
```sql
SELECT * FROM staff_info;
```

These are just basic examples, and you may want to modify them according to your specific requirements.

Sources: ['./data/pdf/mysql-tutorial-excerpt-8.0-en.a4.pdf:16:0', './data/pdf/mysql-tutorial-excerpt-8.0-en.a4.pdf:14:3', './data/pdf/mysql-tutorial-excerpt-8.0-en.a4.pdf:6:1', './data/pdf/mysql-tutorial-excerpt-8.0-en.a4.pdf:14:2', './data/pdf/mysql-tutorial-excerpt-8.0-en.a4.pdf:15:0']